In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
feature_names = [
    "mean_R", "mean_G", "mean_B",
    "std_R", "std_G", "std_B",
    "mean_H", "mean_S", "mean_V",
    "std_H", "std_S", "std_V",
    "dark_ratio",
    "high_saturation_ratio",
    "gray_std",
    "laplacian_var"
]

In [3]:
def extract_features(img, resize_to=(128, 128)):

    img = cv2.resize(img, resize_to)


    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mean_rgb = img_rgb.mean(axis=(0, 1))  
    std_rgb  = img_rgb.std(axis=(0, 1))    

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mean_hsv = img_hsv.mean(axis=(0, 1))   
    std_hsv  = img_hsv.std(axis=(0, 1))    


    H = img_hsv[:, :, 0]
    S = img_hsv[:, :, 1]
    V = img_hsv[:, :, 2]

    dark_ratio = np.mean(V < 50)
    high_saturation_ratio = np.mean(S > 150)

   
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_std = gray.std()
    lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()

   
    features = np.concatenate([
        mean_rgb,       
        std_rgb,           
        mean_hsv,          
        std_hsv,         
        np.array([
            dark_ratio,           
            high_saturation_ratio, 
            gray_std,              
            lap_var               
        ])
    ])

    return features 


In [4]:
def extract_from_folder(images_dir="images", output_csv="new_images_features.csv"):


    if not os.path.isdir(images_dir):
        raise FileNotFoundError(f"المجلد '{images_dir}' غير موجود. أنشئه وضع الصور بداخله.")

    X_list = []
    filenames = []

    print(f"Processing images in folder: {images_dir}")

    for filename in tqdm(sorted(os.listdir(images_dir))):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img_path = os.path.join(images_dir, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"تحذير: تعذر قراءة الصورة: {img_path}")
            continue

        feats = extract_features(img)
        X_list.append(feats)
        filenames.append(filename)

    if len(X_list) == 0:
        raise ValueError("لا توجد صور صالحة في المجلد المحدد.")

    X = np.array(X_list)

    df = pd.DataFrame(X, columns=feature_names)
    df["filename"] = filenames  

    df.to_csv(output_csv, index=False)
    print(f"تم حفظ الميزات في الملف: {output_csv}")
    print(f"عدد الصور المعالجة: {len(filenames)}")
    return df


In [5]:
if __name__ == "__main__":
    extract_from_folder("images", "new_images_features.csv")

Processing images in folder: images


100%|██████████| 6/6 [00:00<00:00, 47.88it/s]

تم حفظ الميزات في الملف: new_images_features.csv
عدد الصور المعالجة: 6
